In [101]:
#some necessary imports
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
from matplotlib import pyplot as plt

In [102]:
#dataset
df = pd.read_csv('../../datasets/train-balanced-sarcasm.csv')
test_df = pd.read_csv('../../datasets/test-balanced.csv')

In [103]:
train_df = df[['label', 'comment']]

In [104]:
df

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...
...,...,...,...,...,...,...,...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the techn...,TwarkMain,reddit.com,2,2,0,2009-04,2009-04-25 00:47:52,"No one is calling this an engineered pathogen,..."
1010822,1,"whatever you do, don't vote green!",BCHarvey,climate,1,1,0,2009-05,2009-05-14 22:27:40,In a move typical of their recent do-nothing a...
1010823,1,Perhaps this is an atheist conspiracy to make ...,rebelcommander,atheism,1,1,0,2009-01,2009-01-11 00:22:57,Screw the Disabled--I've got to get to Church ...
1010824,1,The Slavs got their own country - it is called...,catsi,worldnews,1,1,0,2009-01,2009-01-23 21:12:49,I've always been unsettled by that. I hear a l...


In [105]:
#expections
train_df.head()

,label,comment
0,0,NC and NH.
1,0,You do know west teams play against west teams...
2,0,"They were underdogs earlier today, but since G..."
3,0,"This meme isn't funny none of the ""new york ni..."
4,0,I could use one of those tools.


In [106]:
#some info
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010826 entries, 0 to 1010825
Data columns (total 2 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   label    1010826 non-null  int64 
 1   comment  1010773 non-null  object
dtypes: int64(1), object(1)
memory usage: 15.4+ MB


In [107]:
#some comments are missing, we need to fix it
train_df.dropna(subset=['comment'], inplace=True)

<ipython-input-107-335191bb568a>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [108]:
#this database is balanced
train_df.label.value_counts()

0    505405
1    505368
Name: label, dtype: int64

In [109]:
#split data into train and validation parts
X_train, X_test, y_train, y_test = train_test_split(train_df['comment'], train_df['label'])


In [110]:
#some more imports
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go


In [111]:
## target count ##
cnt_srs = train_df['label'].value_counts()
trace = go.Bar(
    x=cnt_srs.index,
    y=cnt_srs.values,
    marker=dict(
        color=cnt_srs.values,
        colorscale = 'Picnic',
        reversescale = True
    ),
)

layout = go.Layout(
    title='Target Count',
    font=dict(size=18)
)

data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="TargetCount")

## target distribution ##
labels = (np.array(cnt_srs.index))
sizes = (np.array((cnt_srs / cnt_srs.sum())*100))

trace = go.Pie(labels=labels, values=sizes)
layout = go.Layout(
    title='Target distribution',
    font=dict(size=18),
    width=600,
    height=600,
)
data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="usertype")

In [112]:
STOPWORDS = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
from collections import defaultdict
train1_df = train_df[train_df["label"]==1]
train0_df = train_df[train_df["label"]==0]

## custom function for ngram generation ##
def generate_ngrams(text, n_gram=1):
    token = [token for token in text.lower().split(" ") if token != "" if token not in STOPWORDS]
    ngrams = zip(*[token[i:] for i in range(n_gram)])
    return [" ".join(ngram) for ngram in ngrams]

## custom function for horizontal bar chart ##
def horizontal_bar_chart(df, color):
    trace = go.Bar(
        y=df["word"].values[::-1],
        x=df["wordcount"].values[::-1],
        showlegend=False,
        orientation = 'h',
        marker=dict(
            color=color,
        ),
    )
    return trace

## Get the bar chart from sincere questions ##
freq_dict = defaultdict(int)
for sent in train0_df["comment"]:
    for word in generate_ngrams(sent):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace0 = horizontal_bar_chart(fd_sorted.head(50), 'blue')

## Get the bar chart from insincere questions ##
freq_dict = defaultdict(int)
for sent in train1_df["comment"]:
    for word in generate_ngrams(sent):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace1 = horizontal_bar_chart(fd_sorted.head(50), 'blue')

# Creating two subplots
fig = tools.make_subplots(rows=1, cols=2, vertical_spacing=0.04,
                          subplot_titles=["Frequent words of sarcasm phrase",
                                          "Frequent words of unsarcasm phrase"])
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig['layout'].update(height=1200, width=900, paper_bgcolor='rgb(233,233,233)', title="Word Count Plots")
py.iplot(fig, filename='word-plots')

D:\anaconda\lib\site-packages\plotly\tools.py:461: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



In [113]:
freq_dict = defaultdict(int)
for sent in train0_df["comment"]:
    for word in generate_ngrams(sent,2):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace0 = horizontal_bar_chart(fd_sorted.head(50), 'orange')


freq_dict = defaultdict(int)
for sent in train1_df["comment"]:
    for word in generate_ngrams(sent,2):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace1 = horizontal_bar_chart(fd_sorted.head(50), 'orange')

# Creating two subplots
fig = tools.make_subplots(rows=1, cols=2, vertical_spacing=0.04,horizontal_spacing=0.15,
                          subplot_titles=["Frequent bigrams of sarcasm phrase",
                                          "Frequent bigrams of unsarcasm phrase"])
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig['layout'].update(height=1200, width=900, paper_bgcolor='rgb(233,233,233)', title="Bigram Count Plots")
py.iplot(fig, filename='word-plots')

In [114]:
freq_dict = defaultdict(int)
for sent in train0_df["comment"]:
    for word in generate_ngrams(sent,3):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace0 = horizontal_bar_chart(fd_sorted.head(50), 'green')


freq_dict = defaultdict(int)
for sent in train1_df["comment"]:
    for word in generate_ngrams(sent,3):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace1 = horizontal_bar_chart(fd_sorted.head(50), 'green')

# Creating two subplots
fig = tools.make_subplots(rows=1, cols=2, vertical_spacing=0.04, horizontal_spacing=0.2,
                          subplot_titles=["Frequent trigrams of sarcasm phrase",
                                          "Frequent trigrams of unsarcasm phrase"])
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig['layout'].update(height=1200, width=1200, paper_bgcolor='rgb(233,233,233)', title="Trigram Count Plots")
py.iplot(fig, filename='word-plots')

In [115]:
X_train = pd.DataFrame(data={'comment': X_train})
X_test = pd.DataFrame(data={'comment': X_test})

In [116]:
X_test

,comment
153651,"""Just a smack on the face from a truck, I'm su..."
939510,"Fuck off, semen breath."
901808,"Not before giving each of them a warm hug, I'm..."
563942,One of us One of us
705700,I can't.
...,...
949098,"I run 2 and a half miles every day, lift on ev..."
326077,Because if the power is from ownership the Fre...
814776,"minority here, GG turned me into a bigot and n..."
176673,All the cool kids are doing it.


In [117]:
import string

X_train["num_words"] = X_train["comment"].apply(lambda x: len(str(x).split()))
X_test["num_words"] = X_test["comment"].apply(lambda x: len(str(x).split()))

## Number of unique words in the text ##
X_train["num_unique_words"] = X_train["comment"].apply(lambda x: len(set(str(x).split())))
X_test["num_unique_words"] = X_test["comment"].apply(lambda x: len(set(str(x).split())))

## Number of characters in the text ##
X_train["num_chars"] = X_train["comment"].apply(lambda x: len(str(x)))
X_test["num_chars"] = X_test["comment"].apply(lambda x: len(str(x)))

## Number of stopwords in the text ##
X_train["num_stopwords"] = X_train["comment"].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))
X_test["num_stopwords"] = X_test["comment"].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))

## Number of punctuations in the text ##
X_train["num_punctuations"] =X_train['comment'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
X_test["num_punctuations"] =X_test['comment'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )

## Number of title case words in the text ##
X_train["num_words_upper"] = X_train["comment"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
X_test["num_words_upper"] = X_test["comment"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

## Number of title case words in the text ##
X_train["num_words_title"] = X_train["comment"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
X_test["num_words_title"] = X_test["comment"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))

## Average length of the words in the text ##
X_train["mean_word_len"] = X_train["comment"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
X_test["mean_word_len"] = X_test["comment"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [120]:
tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1,3))
tfidf_vec.fit_transform(X_train['comment'].values.tolist() + X_test['comment'].values.tolist())
train_tfidf = tfidf_vec.transform(X_train['comment'].values.tolist())
test_tfidf = tfidf_vec.transform(X_test['comment'].values.tolist())

In [130]:
train_df = X_train.merge(y_train, left_index=True, right_index=True)

In [136]:
train_df.isnull().sum()

comment             0
num_words           0
num_unique_words    0
num_chars           0
num_stopwords       0
num_punctuations    0
num_words_upper     0
num_words_title     0
mean_word_len       0
label               0
dtype: int64

In [140]:
from sklearn import model_selection, metrics
X_train.dropna()
model = LogisticRegression(C=5., solver='sag')
model.fit(X_train, y_train)
pred_test_y = model.predict_proba(X_test)[:,1]


ValueError: could not convert string to float: 'Cutest nightmare fuel ever.'

In [141]:
X_train

,comment,num_words,num_unique_words,num_chars,num_stopwords,num_punctuations,num_words_upper,num_words_title,mean_word_len
644897,Cutest nightmare fuel ever.,4,4,27,0,1,0,1,6.000000
34270,He's my fancast for Black Racer,6,6,31,2,1,0,2,4.333333
994467,you mad you ain't in teh special club bro?,9,8,42,3,2,0,0,3.777778
516681,"Ahahaha, because women never say what they wan...",34,30,186,18,6,0,1,4.500000
252279,$7 for 20 damn what year was that,8,8,33,4,1,0,0,3.250000
...,...,...,...,...,...,...,...,...,...
993685,"I can't find the like button, guess an upvote ...",13,13,62,6,3,1,1,3.846154
967717,BRAAAAAAAAINS!,1,1,14,0,1,1,0,14.000000
683666,First!,1,1,6,0,1,0,1,6.000000
595864,Thanks for speaking up sooner.,5,5,30,2,1,0,1,5.200000


,comment,num_words,num_unique_words,num_chars,num_stopwords,num_punctuations,num_words_upper,num_words_title,mean_word_len,label
644897,Cutest nightmare fuel ever.,4,4,27,0,1,0,1,6.000000,0
34270,He's my fancast for Black Racer,6,6,31,2,1,0,2,4.333333,1
994467,you mad you ain't in teh special club bro?,9,8,42,3,2,0,0,3.777778,1
516681,"Ahahaha, because women never say what they wan...",34,30,186,18,6,0,1,4.500000,1
252279,$7 for 20 damn what year was that,8,8,33,4,1,0,0,3.250000,0
...,...,...,...,...,...,...,...,...,...,...
993685,"I can't find the like button, guess an upvote ...",13,13,62,6,3,1,1,3.846154,1
967717,BRAAAAAAAAINS!,1,1,14,0,1,1,0,14.000000,0
683666,First!,1,1,6,0,1,0,1,6.000000,0
595864,Thanks for speaking up sooner.,5,5,30,2,1,0,1,5.200000,1
